In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('../data/G_train.csv')
test = pd.read_csv('../data/G_test.csv')

In [4]:
user2idx = {user:idx for idx, user in enumerate(train['user_code'].unique())}
train['user_idx'] = train['user_code'].map(user2idx)
idx2user = {idx:user for idx, user in enumerate(train['user_code'].unique())}

In [5]:
item2idx = {item:idx for idx, item in enumerate(train['rest_code'].unique())}
train['rest_idx'] = train['rest_code'].map(item2idx)
idx2item = {idx:item for idx, item in enumerate(train['rest_code'].unique())}

In [6]:
test_user2idx = {user:idx for idx, user in enumerate(test['user_code'].unique())}
test['user_idx'] = test['user_code'].map(user2idx)
test_idx2user = {idx:user for idx, user in enumerate(test['user_code'].unique())}

In [7]:
test_item2idx = {item:idx for idx, item in enumerate(test['rest_code'].unique())}
test['rest_idx'] = test['rest_code'].map(item2idx)
test_idx2item = {idx:item for idx, item in enumerate(test['rest_code'].unique())}

In [8]:
train

,userid,rest,user_code,rest_code,user_idx,rest_idx
0,5b61c7658f8242cb2a1b1028,1.172647e+07,166651,2073,0,0
1,5b61c7658f8242cb2a1b1028,2.140700e+07,166651,1604,0,1
2,5b61c7658f8242cb2a1b1028,1.176442e+07,166651,1160,0,2
3,5b61c7658f8242cb2a1b1028,1.987767e+07,166651,1571,0,3
4,5b61c7658f8242cb2a1b1028,1.100141e+09,166651,658,0,4
...,...,...,...,...,...,...
338124,6396b0ffadfbb231c28da55d,1.551480e+09,320304,2235,32681,1796
338125,6396b0ffadfbb231c28da55d,1.016763e+09,320304,2166,32681,1655
338126,6396b0ffadfbb231c28da55d,2.062259e+07,320304,2324,32681,2310
338127,6396b0ffadfbb231c28da55d,1.357503e+07,320304,2100,32681,1307


In [11]:
def generate_rating_matrix_submission(user_seq, num_users, num_items):
    # three lists are used to construct sparse matrix
    """
    Args:
        user_seq (2차원 list): [[1번 유저 item_id 리스트], [2번 유저 item_id 리스트] .. ]
        num_users (int): 유저 수
        num_items (int): 아이템 수(정확힌 max item_id)
    Returns:
        rating_matrix: 크기 (num_users, num_items) 유저-아이템 행렬, 유저의 영화시청기록 빼지 않음.
    """ 
    row = [] # user_id가 담긴 리스트
    col = [] # 유저 별 item_id 리스트가 담긴 리스트
    data = [] # 1이 달린(positive sampling이라고 알려주는) 리스트.

    # user_id : 유저 번호, item_list : 해당 유저 item_id list
    for user_id, item_list in enumerate(user_seq):
        for item in item_list[:]: # 해당 유저가 시청한 영화기록 제외하지 않고 모두 포함.
            row.append(user_id)
            col.append(item)
            data.append(1)

    # 리스트를 넘파이 array로 바꿔줍니다.
    row = np.array(row)
    col = np.array(col) # 이 때 2차원 리스트는 겉만 np.array로 바뀌고 속은 list를 유지합니다.
    data = np.array(data)

    # 희소행렬 메트릭스 연산을 도와주는 scipy 내 csr_matrix 함수를 이용해 유저-아이템 행렬 제작합니다.
    rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))

    return rating_matrix

In [12]:
def get_user_seqs(train, test):
    """
    Args:
        train : train 데이터 user/item csv 파일.
        test : test t데이터 user/item csv 파일.

    Returns:
        user_seq : train 유저마다 따로 아이템 리스트 저장. 2차원 배열.
        => [[1번 유저 item_id 리스트], [2번 유저 item_id 리스트] .. ]
        test_user_seq : test 유저마다 따로 아이템 리스트 저장. 2차원 배열.
        => [[1번 유저 item_id 리스트], [2번 유저 item_id 리스트] .. ]
        max_item : 가장 큰 item_id, item의 개수를 나타냄.
        train_matrix : 유저-아이템 희소행렬
    """    

    # lines : 유저인덱스/아이템리스트 형식의 판다스가 나옵니다.
    # ex) 11 [4643, 170, 531, 616, 2140, 2722, 2313, 2688, ...]
    train_lines = train.groupby("user_idx")["rest_idx"].apply(list)
    test_lines = test.groupby("user_idx")["rest_idx"].apply(list)

    # user_seq : 유저마다 따로 아이템 리스트 저장. 2차원 배열.
    # ex) [[1번 유저 item_id 리스트], [2번 유저 item_id 리스트] .. ]
    user_seq = []
    test_user_seq = []
    
    item_set = set()

    for line in train_lines: # line : 한 유저의 아이템 리스트
        items = line
        user_seq.append(items) # append : 리스트를 하나의 원소로 보고 append함
        item_set = item_set | set(items) # | : 합집합 연산자

    for line in test_lines: # line : 한 유저의 아이템 리스트
        items = line
        test_user_seq.append(items) # append : 리스트를 하나의 원소로 보고 append함

    # 기록된 가장 큰 아이템 id(번호)
    max_item = max(item_set)
    # len(lines) : 유저 수.
    num_users = len(train_lines)
    # num_items : 가장 큰 아이템 id를 기준으로 아이템 수 측정
    num_items = max_item + 2


    # train_matrix : 유저-아이템 희소행렬
    train_matrix = generate_rating_matrix_submission(
        user_seq, num_users, num_items
    )
    return (
        user_seq,
        test_user_seq,
        max_item,
        train_matrix,
    )

In [14]:
user_seq, test_user_seq, max_item, train_matrix = get_user_seqs(train, test)

In [19]:
from sklearn.metrics.pairwise import linear_kernel

In [20]:
cosine_sim = linear_kernel(train_matrix, train_matrix)

In [62]:
def get_similar_user(user_idx: int, cosine_sim):
    sim_scores = list(enumerate(cosine_sim[user_idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    user_indices = [i[0] for i in sim_scores]
    return user_indices

In [115]:
def get_recommend_list(user_idx: int, cosine_sim, matrix):
    
    sim_user_list = get_similar_user(user_idx=user_idx, cosine_sim=cosine_sim)

    sim_matrix = matrix[user_idx].toarray()# * -10

    sim_matrix = np.vstack([sim_matrix * -10, matrix[sim_user_list].toarray()])
        
    sim_matrix_sum = sim_matrix.sum(axis=0)

    topid = list(reversed(sorted(range(len(sim_matrix_sum)),key= lambda i: sim_matrix_sum[i])[-3:]))

    result = []

    for t in topid:
        result.append(idx2item[t])

    return result

In [39]:
test.iloc[58691,5] = 2758

In [41]:
data = pd.concat([train,test])

In [44]:
data = data.sort_values(by='user_idx')

In [53]:
data['rest_idx'] = data['rest_idx'].astype(int)

In [54]:
data_lines = data.groupby("user_idx")["rest_idx"].apply(list)

data_user_seq = []
item_set = set()

for line in data_lines: # line : 한 유저의 아이템 리스트
        items = line
        data_user_seq.append(items) # append : 리스트를 하나의 원소로 보고 append함
        item_set = item_set | set(items) # | : 합집합 연산자

# 기록된 가장 큰 아이템 id(번호)
max_item = max(item_set)
# len(lines) : 유저 수.
num_users = len(data_lines)
# num_items : 가장 큰 아이템 id를 기준으로 아이템 수 측정
num_items = max_item + 2

data_matrix = generate_rating_matrix_submission(
        data_user_seq, num_users, num_items
    )

In [73]:
data_cos_sim = linear_kernel(data_matrix, data_matrix)

In [75]:
get_recommend_list(user_idx=0, cosine_sim=data_cos_sim, matrix=data_matrix)

[594, 7, 23]

In [86]:
get_similar_user(user_idx=0, cosine_sim=cosine_sim)

[8175, 10184, 10739, 11700, 18171, 21007, 23394, 25440, 22, 105]

In [116]:
get_recommend_list(user_idx=0, cosine_sim=cosine_sim, matrix=train_matrix)

[1558, 970, 7]

In [130]:
_test = test.groupby('user_idx')['rest_code'].unique().to_frame().reset_index()
_test['pred'] = _test['user_idx'].apply(lambda x : get_recommend_list(user_idx=x, cosine_sim=cosine_sim, matrix=train_matrix))

In [ ]:
_test2 = _test.iloc[:300, :]
_test2

In [123]:
def recallk(actual, predicted, k = 3):
    """ label과 prediction 사이의 recall 평가 함수 
    Args:
        actual : 실제로 본 상품 리스트
        pred : 예측한 상품 리스트
        k : 상위 몇개의 데이터를 볼지 (ex : k=5 상위 5개의 상품만 봄)
    Returns: 
        recall_k : recall@k 
    """ 
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

In [131]:
_test['recall'] = _test.apply(lambda x : recallk(x['rest_code'], x['pred']), axis = 1)

In [132]:
_test['recall'].mean()

0.05524447708218591